In [30]:
import openai
import json
import csv
import pandas as pd
import numpy as np
import time
import os

In [31]:
from openai import AsyncOpenAI
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
client = AsyncOpenAI(api_key=api_key)

In [32]:
async def create_response(model, input, response_schema, system_prompt):
    response = await client.responses.create(
        model=model,
        input=input,
        instructions=system_prompt,
        text={
            "format": {
                "name": "statement_response",
                "type": "json_schema",
                "strict": True,
                "schema": response_schema
            }
        }
    )
    return response

In [33]:
def construct_response_schema(response_options):
    return {
        "type": "object",
        "properties": {
            "input_statement": {
                "type": "string",
                "description": "The exact statement being responded to"
            },
            "response": {
                "type": "string",
                "description": "The response to the statement",
                "enum": response_options
            }
        },
        "required": ["input_statement", "response"],
        "additionalProperties": False
    }

In [34]:
def construct_generic_system_prompt(language, ai_definition, response_options):
    return f"""
    You are a primary or secondary school teacher. You have to respond to a survey statement in {language}.
    Please indicate your level of agreement with the statement about the use of artificial intelligence (AI) in education.
    In the survey, AI is defined as below in {language}:
    {ai_definition}

    Please respond with only one of the response options for each statement.
    Response options:
    {response_options}
    """

In [35]:
def verify_response_match_options(response, response_options):
    """Check if response is in the list of valid options. Returns bool."""
    return response in response_options

In [6]:
def construct_generic_user_input(statement):
    return f"Statement to respond to: {statement}"


In [36]:
def save_json_file(file_path, data):
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)


In [42]:
def save_response(response_json):
    """Save full LLM response to responses/ folder.
    
    Args:
        response_json: The full response object as a dictionary
        
    Returns:
        str: The response ID
    """
    response_id = response_json["id"]
    filepath = f"output/responses/{response_id}.json"
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(response_json, f, indent=2, ensure_ascii=False)
    return response_id

In [38]:
def append_to_csv(filepath, row_data):
    """Append a single row to CSV, create with headers if file doesn't exist.
    
    Args:
        filepath: Path to the CSV file
        row_data: Dictionary containing row data
    """
    fieldnames = ['question_id', 'run_number', 'attempt', 'response_from_llm', 'response_score', 'llm_response_id']
    file_exists = os.path.exists(filepath)
    
    with open(filepath, 'a', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerow(row_data)

In [39]:
async def process_statement(statement, run_number, response_options, response_options_obj, 
                            response_schema, system_prompt, csv_filepath):
    """Process a single statement with retry logic.
    
    Args:
        statement: Statement dict with 'id' and 'prompt'
        run_number: Current run iteration number
        response_options: List of valid response options
        response_options_obj: Dict mapping responses to scores
        response_schema: JSON schema for response validation
        system_prompt: System prompt for the LLM
        csv_filepath: Path to CSV file for logging
        
    Returns:
        bool: True if successful, False if all attempts failed
    """
    question_id = statement["id"]
    user_input = construct_generic_user_input(statement["prompt"])
    
    for attempt in range(1, MAX_ATTEMPTS + 1):
        response = await create_response(MODEL, user_input, response_schema, system_prompt)
        response_json = response.model_dump()
        output = json.loads(response_json["output"][0]["content"][0]["text"])
        
        # Save full response to responses/ folder
        response_id = save_response(response_json)
        
        # Validate response
        if verify_response_match_options(output["response"], response_options):
            response_score = response_options_obj[output["response"]]
            append_to_csv(csv_filepath, {
                'question_id': question_id,
                'run_number': run_number,
                'attempt': attempt,
                'response_from_llm': output["response"],
                'response_score': response_score,
                'llm_response_id': response_id
            })
            return True
        
        print(f"Invalid response '{output['response']}' for {question_id}, attempt {attempt}/{MAX_ATTEMPTS}")
    
    print(f"FAILED: {question_id} after {MAX_ATTEMPTS} attempts")
    return False

In [44]:
async def process_file(filepath):
    """Process all statements in a file for NUM_RUNS iterations.
    
    Args:
        filepath: Path to the questions JSON file
    """
    content = json.load(open(filepath, encoding='utf-8'))
    
    # Extract file name for CSV (e.g., "questions/usa_english.json" -> "generic/usa_english.csv")
    filename = os.path.basename(filepath).replace('.json', '.csv')
    csv_filepath = f"output/generic/{filename}"
    
    # Clear existing CSV for fresh run
    if os.path.exists(csv_filepath):
        os.remove(csv_filepath)
    
    # Setup from file content
    language = content["language"]
    ai_definition = content["questions"][0]["description"]
    response_options_obj = content["questions"][0]["responses"]
    response_options = list(response_options_obj.keys())
    response_schema = construct_response_schema(response_options)
    system_prompt = construct_generic_system_prompt(language, ai_definition, response_options)
    statements = content["questions"][0]["questions"]
    
    total_statements = len(statements)
    
    # Run NUM_RUNS times
    for run_number in range(1, NUM_RUNS + 1):
        print(f"[{filepath}] Run {run_number}/{NUM_RUNS}")
        for idx, statement in enumerate(statements, 1):
            print(f"  Processing statement {idx}/{total_statements}: {statement['id']}", end=" ")
            success = await process_statement(
                statement, run_number, response_options, response_options_obj,
                response_schema, system_prompt, csv_filepath
            )
            print("✓" if success else "✗")
    
    print(f"Completed: {filepath} -> {csv_filepath}")

In [46]:
# =============================================================================
# CONFIGURATION
# =============================================================================

MODEL = "gpt-4o"
MAX_ATTEMPTS = 3  # Maximum retry attempts for invalid responses
NUM_RUNS = 2      # Number of times to run each file

FILES = [
    # "questions/usa_english.json",
    "questions/alb_albanian.json",
]

for filepath in FILES:
    await process_file(filepath)


[questions/alb_albanian.json] Run 1/2
  Processing statement 1/10: TT4G35A ✓
  Processing statement 2/10: TT4G35B ✓
  Processing statement 3/10: TT4G35C ✓
  Processing statement 4/10: TT4G35D ✓
  Processing statement 5/10: TT4G35E ✓
  Processing statement 6/10: TT4G35F ✓
  Processing statement 7/10: TT4G35G ✓
  Processing statement 8/10: TT4G35H ✓
  Processing statement 9/10: TT4G35I ✓
  Processing statement 10/10: TT4G35J ✓
[questions/alb_albanian.json] Run 2/2
  Processing statement 1/10: TT4G35A ✓
  Processing statement 2/10: TT4G35B ✓
  Processing statement 3/10: TT4G35C ✓
  Processing statement 4/10: TT4G35D ✓
  Processing statement 5/10: TT4G35E ✓
  Processing statement 6/10: TT4G35F ✓
  Processing statement 7/10: TT4G35G ✓
  Processing statement 8/10: TT4G35H ✓
  Processing statement 9/10: TT4G35I ✓
  Processing statement 10/10: TT4G35J ✓
Completed: questions/alb_albanian.json -> output/generic/alb_albanian.csv
